## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt5


nx = 9
ny = 6

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

def calibration(images):
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        img_size = (img.shape[1], img.shape[0])
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
    return mtx, dist

cam_mtx, cam_dist = calibration(images)

        #cv2.imshow('img',img)
        #cv2.waitKey(500)

#cv2.destroyAllWindows()


In [2]:
def mask_image(image):
    sw_x = image.shape[1] * .01
    sw_y = image.shape[0]
    nw_x = int(image.shape[1] * .35)
    nw_y = int(image.shape[0] * .65)
    ne_x = int(image.shape[1] * .65)
    ne_y = int(image.shape[0] * .65)
    se_x = int(image.shape[1] + (image.shape[1] * .1))
    se_y = int(image.shape[0])


    vertices = np.array([[(sw_x, sw_y),
                          (nw_x,nw_y),
                          (ne_x,ne_y),
                          (se_x,se_y)]], dtype=np.int32)



    mask = np.zeros_like(image)
    # defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    # filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    # returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def source():
    src = np.float32([
        [475,530],
        [830,530],
        [130,720],
        [1120,720]
    ])
    return src

def destination():
    src = np.float32([
        [365,540],
        [990,540],
        [320,720],
        [960,720]
    ])
    return src

## And so on and so forth...

In [3]:
BIN_256 = 256
HIST_RANGE_MIN = 0
HIST_RANGE_MAX = 256
MASKED_ARRAY_MIN = 0
MASKED_ARRAY_MAX = 255

def increase_contrast(image):

    equ = cv2.equalizeHist(image)
    hist, bins = np.histogram(equ.flatten(), BIN_256, [HIST_RANGE_MIN, HIST_RANGE_MAX])
    cdf = hist.cumsum()
    cdf_m = np.ma.masked_equal(cdf, MASKED_ARRAY_MIN)
    cdf_m = (cdf_m - cdf_m.min()) * MASKED_ARRAY_MAX / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, MASKED_ARRAY_MIN).astype('uint8')
    histogram = cdf[image]
    return cv2.equalizeHist(histogram)
def get_threshold(image):
    hls = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2HLS)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    gray = increase_contrast(gray)
    s = hls[:, :, 2]
    x, gray_threshold = cv2.threshold(gray.astype('uint8'), 75, 255, cv2.THRESH_BINARY)
    x, s_threshold = cv2.threshold(s.astype('uint8'), 75, 255, cv2.THRESH_BINARY)
    combined_binary = np.clip(cv2.bitwise_and(gray_threshold, s_threshold), 0, 1).astype('uint8')
    return combined_binary

In [4]:
image = cv2.imread('../test_images/test1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(image,cmap='gray')
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(undistorted_dash)
ax2.set_title('Undistorted Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
cv2.imwrite('Undistorted Image.jpg',undistorted_dash)

True

In [5]:
image = cv2.imread('../test_images/test2.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=12)
ax2.imshow(warped)
ax2.set_title('warped Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
cv2.imwrite('warped Image.jpg',warped)

True

In [11]:
image = cv2.imread('../test_images/test3.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)
pipe = get_threshold(warped)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
f.tight_layout()
ax1.imshow(warped)
ax1.set_title('Undistorted Image', fontsize=12)
ax2.imshow(pipe,cmap='gray')
ax2.set_title('Threshold Image', fontsize=12)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
cv2.imwrite('Threshold Image.jpg',pipe)

True

In [12]:
image = cv2.imread('../test_images/test3.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
masked = mask_image(image)
undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
m = cv2.getPerspectiveTransform(source(), destination())
m_inverse = cv2.getPerspectiveTransform(destination(), source())
image_size = (undistorted_dash.shape[1], undistorted_dash.shape[0])
warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)
pipe = get_threshold(warped)
histogram = np.sum(pipe[pipe.shape[0] // 2:, :], axis=0)
out_img = np.dstack((pipe, pipe, pipe)) * 255
#
plt.plot(histogram)
plt.show(block=True)

In [13]:
# Assuming you have created a warped binary image called "pipe"
# Take a histogram of the bottom half of the image
histogram = np.sum(pipe[pipe.shape[0]//2:,:], axis=0)
# Create an output image to draw on and  visualize the result
out_img = np.dstack((pipe, pipe, pipe))*255

# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(pipe.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = pipe.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = pipe.shape[0] - (window+1)*window_height
    win_y_high = pipe.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),
    (0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),
    (0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
    (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, pipe.shape[0]-1, pipe.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)
cv2.imwrite('plot Image.jpg',out_img)

True

In [14]:
nonzero = pipe.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
margin = 100
left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
left_fit[1]*nonzeroy + left_fit[2] + margin))) 

right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
right_fit[1]*nonzeroy + right_fit[2] + margin)))  

# Again, extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds]
# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)
# Generate x and y values for plotting
ploty = np.linspace(0, pipe.shape[0]-1, pipe.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
# Create an image to draw on and an image to show the selection window
out_img = np.dstack((pipe, pipe, pipe))*255
window_img = np.zeros_like(out_img)
# Color in left and right line pixels
out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)
cv2.imwrite('plot.jpg',result)

True

In [15]:
undist = cv2.undistort(image, cam_mtx, cam_dist, None, cam_mtx)

# Create an image to draw the lines on
warp_zero = np.zeros_like(pipe).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(warped, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix
newwarp = cv2.warpPerspective(warped, m_inverse, (image.shape[1], image.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
plt.imshow(result)
cv2.imwrite('result.jpg',result)

True

In [8]:
YM_PER_PX = 30 / 720
XM_PER_PX = 3.7 / 700

#calculate curves
def calculate_curves(leftx,lefty,rightx,righty):
    left_fit_cr = np.polyfit(lefty * YM_PER_PX, leftx * XM_PER_PX, 2)
    right_fit_cr = np.polyfit(righty * YM_PER_PX, rightx * XM_PER_PX, 2)

    left_curverad = ((1 + (2 * left_fit_cr[0] * np.max(lefty) + left_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * left_fit_cr[0])
    right_curverad = ((1 + (2 * right_fit_cr[0] * np.max(lefty) + right_fit_cr[1]) ** 2) ** 1.5) / np.absolute(2 * right_fit_cr[0])
    return left_curverad,right_curverad

def get_center_calc(image,left_fitx,right_fitx):
    vehicle_position = image.shape[1] / 2
    center_of_lane = (left_fitx[-1] + right_fitx[-1]) // 2
    center_calc = (vehicle_position - center_of_lane) * XM_PER_PX
    return center_calc


class Lanes:
    cached_result = None
    cached_lane = None
    last_good_right_curve = None
    last_good_left_curve = None

In [9]:
def pipevi(image):
    masked = mask_image(image)
    undistorted_dash = cv2.undistort(masked, cam_mtx, cam_dist, None, cam_mtx)
    m = cv2.getPerspectiveTransform(source(), destination())
    m_inverse = cv2.getPerspectiveTransform(destination(),source())
    image_size = (undistorted_dash.shape[1],undistorted_dash.shape[0])
    warped = cv2.warpPerspective(undistorted_dash, m, image_size, flags=cv2.INTER_LINEAR)
    pipe = get_threshold(warped)

    histogram = np.sum(pipe[pipe.shape[0]//2:,:], axis=0)
    out_img = np.dstack((pipe, pipe, pipe))*255

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    window_height = np.int(pipe.shape[0]/nwindows)
    nonzero = pipe.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 100
    minpix = 50
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = pipe.shape[0] - (window+1)*window_height
        win_y_high = pipe.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2)
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2)
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

# this try catch culls out bad frames
    try:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        # Fit a second order polynomial to each
        Lanes.last_good_right = right_fit
    except:
        return Lanes.cached_result

    ploty = np.linspace(0, pipe.shape[0]-1, pipe.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    #
    # out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    # out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    ####
    # calculate curves and centerline
    left_curverad, right_curverad =  calculate_curves(leftx, lefty, rightx, righty)
    center_calc = get_center_calc(image,left_fitx,right_fitx)

    ###
    warp_zero = np.zeros_like(pipe).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # this is a hamfisted solution to replace curves with extreme values with the previous frame's curve
    if (right_curverad > 5000 or left_curverad > 5000) and Lanes.cached_lane is not None:
        newwarp = Lanes.cached_lane
    else:
        newwarp = cv2.warpPerspective(color_warp, m_inverse, (image.shape[1], image.shape[0]))
        Lanes.cached_lane = newwarp
        Lanes.last_good_right_curve = right_curverad
        Lanes.last_good_left_curve = left_curverad

    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    Lanes.cached_result = result
    
    text = 'curvature radius: {0} m. '.format((int(left_curverad) + int(right_curverad)) / 2 )
    text2 = 'distance from center: {0} m. '.format(( np.math.ceil(abs(center_calc) * 100) / 100))
    cv2.putText(result,text,(25,75),cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,0),2,cv2.LINE_AA)
    cv2.putText(result, text2, (25, 120), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 0), 2, cv2.LINE_AA)
    return result

In [16]:
plt.imshow(pipevi(image))
cv2.imwrite('result1.jpg',pipevi(image))

True

In [18]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

write_output = '../output_images/project_video_out.mp4'
clip1 = VideoFileClip("../project_video.mp4")
write_clip = clip1.fl_image(pipevi) #NOTE: this function expects color images!!
%time write_clip.write_videofile(write_output, audio=False)

[MoviePy] >>>> Building video ../output_images/project_video_out.mp4
[MoviePy] Writing video ../output_images/project_video_out.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [05:49<00:00,  3.83it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output_images/project_video_out.mp4 

Wall time: 5min 52s
